<a href="https://colab.research.google.com/github/hyesukim1/Face-Verification-Project/blob/main/facemaskdetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
ls -1ha kaggle.json

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d andrewmvd/face-mask-detection

In [ ]:
!ls

In [ ]:
!unzip -qq "/content/face-mask-detection.zip"

In [ ]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import torchvision
from torchvision import transforms, datasets, models
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import matplotlib.patches as patches
import os
from torch.utils.data import DataLoader

In [ ]:
def generate_box(obj):
  xmin = int(obj.find('xmin').text)
  ymin = int(obj.find('ymin').text)
  xmax = int(obj.find('xmax').text)
  ymax = int(obj.find('ymax').text)
  return [xmin, ymin, xmax, ymax]

def generate_label(obj):
  if obj.find('name').text == 'with_mask':
    return 1
  elif obj.find('name').text == 'mask_weared_incorrect':
    return 2
  return 0

def generate_target(image_id, file):
  with open(file) as f:
    data = f.read()
    soup = BeautifulSoup(data, 'xml')
    objects = soup.find_all('object')

    num_objs = len(objects)

    # bounding boxes for objects
    boxes = []
    labels = []

    for i in objects:
      boxes.append(generate_box(i))
      labels.append(generate_label(i))
    
    # boxes, labels to tensor
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    labels = torch.as_tensor(labels, dtype=torch.int64)
    
    #tensorise image id
    img_id = torch.tensor([image_id])

    target = {}
    target['boxes'] = boxes
    target['labels'] = labels
    target['image_id'] = img_id

    return target


In [ ]:
images = list(sorted(os.listdir("/content/images/")))
labels = list(sorted(os.listdir("/content/annotations/")))

In [ ]:
class MaskDataset(object):
  def __init__(self, transforms):
    self.transforms = transforms
    # load all images files, because of sorting them that they are aligned
    self.imgs = list(sorted(os.listdir("/content/images/")))

  def __len__(self):
      return len(self.imgs)

  def __getitem__(self, idx):
    file_image = 'maksssksksss' + str(idx) + '.png'
    file_label = 'maksssksksss' + str(idx) + '.xml'
    img_path = os.path.join("/content/images/" , file_image)
    label_path = os.path.join("/content/annotations/" , file_label)

    img = Image.open(img_path).convert("RGB")
    target = generate_target(idx, label_path)

    if self.transforms is not None:
      img = self.transforms(img)

    return img, target

In [ ]:
data_transform = transforms.Compose([transforms.ToTensor()])

In [ ]:
def collate_fn(batch):
  return tuple(zip(*batch))

dataset = MaskDataset(data_transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=4, collate_fn=collate_fn)

In [ ]:
print(data_loader)

In [ ]:
torch.cuda.is_available()

# Make model & train

In [ ]:
def get_model_instance_segmentation(num_classes):
  # coco dataset에서 미리 학습된 인스턴스 분할 모델 읽어오기
  model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
  # 분류를 위한 입력 특징 차원을 얻음
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  # 미리 학습된 헤더를 새로운 것으로 바꿈
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

  return model

In [ ]:
# 3개 클래스로 분류하는 모델
model = get_model_instance_segmentation(3)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
for imgs, annotations in data_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)
    break

In [ ]:
num_epochs = 25
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

len_detaloader = len(data_loader)
print(len_detaloader)
# print(params)
# print(optimizer)

In [ ]:
for epoch in range(num_epochs):
  model.train()
  i = 0
  epoch_loss = 0
  for imgs, annotations in data_loader:
    i += 1
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    loss_dict = model([imgs[0]], [annotations[0]])
    losses = sum(loss for loss in loss_dict.values())

    optimizer.zero_grad()
    losses.backward()
    optimizer.step()

    epoch_loss += losses

  print(epoch_loss)

In [ ]:
for imgs, annotations in data_loader:
  imgs = list(img.to(device) for img in imgs)
  annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
  break

In [ ]:
model.eval()
preds = model(imgs)
print(preds[2]['boxes'])

# show images

In [ ]:
def plot_image(img_tensor, annotation):

  fig,ax = plt.subplots(1)
  img = img_tensor.cpu().data

  ax.imshow(img.permute(1, 2, 0))

  for box in annotation["boxes"].cpu().data.numpy():
    xmin, ymin, xmax, ymax = box

    rect = patches.Rectangle((xmin, ymin), (xmax-xmin), (ymax-ymin), linewidth=1, edgecolor='r', facecolor='none')

    ax.add_patch(rect)

  plt.show()

In [ ]:
print("Prediction")
plot_image(imgs[2], preds[2])
print("Target")
plot_image(imgs[2], annotations[2])

# save model

In [ ]:
torch.save(model.state_dict(), 'model.pt')
model2 = get_model_instance_segmentation(3)

In [ ]:
model2.load_state_dict(torch.load('model.pt'))
model2.eval()
model2.to(device)

# Load Model

In [ ]:
predict2 = model2(imgs)

In [ ]:
print("prediction with loaded model")
plot_image(imgs[3], predict2[3])